In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
#langchain-imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# Vector stores
from langchain_community.vectorstores import Chroma

# Utility imports
import numpy as np
from typing import List


c:\RAG Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. Sample Data

In [3]:
## create sample documents
sample_docs = [
    """
    Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    """,
    
    """
    Deep Learning and Neural Networks
    
    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning has revolutionized fields like computer vision, natural language 
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly 
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers 
    excel at sequential data processing.
    """,
    
    """
    Natural Language Processing (NLP)
    
    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.
    """
]

sample_docs

['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\n    ',
 '\n    Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective f

In [5]:
# Save sample documents to files
import tempfile
tempdir =  tempfile.mkdtemp()

for i,doc in enumerate(sample_docs):
    with open(f"{tempdir}/doc_{i}.txt","w") as f:
        f.write(doc)
    print(f"sample docs created in {tempdir}")

sample docs created in C:\Users\vvsad\AppData\Local\Temp\tmpxb2vrom9
sample docs created in C:\Users\vvsad\AppData\Local\Temp\tmpxb2vrom9
sample docs created in C:\Users\vvsad\AppData\Local\Temp\tmpxb2vrom9


In [7]:
# Save sample documents to files
import tempfile
tempdir =  tempfile.mkdtemp()

for i,doc in enumerate(sample_docs):
    with open(f"doc_{i}.txt","w") as f:
        f.write(doc)
    print(f"sample docs created in {tempdir}")

sample docs created in C:\Users\vvsad\AppData\Local\Temp\tmpz1vz8s1k
sample docs created in C:\Users\vvsad\AppData\Local\Temp\tmpz1vz8s1k
sample docs created in C:\Users\vvsad\AppData\Local\Temp\tmpz1vz8s1k


2. Document Loading

In [ ]:
import glob
from langchain_community.document_loaders import TextLoader, DirectoryLoader

# load documents from directory
loader = DirectoryLoader(
    "data",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
# load documents
docs = loader.load()
print(f"Loaded {len(docs)} documents")
print("\nFirst document: ")
print(docs[0].page_content)

Loaded 3 documents

First document: 

    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    


### 3. Document Splitting

In [11]:
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # Maximum number of characters per chunk
    chunk_overlap=50, # Overlap between chunks to maintain context
    length_function=len, # Function to calculate the length of the text
    separators=[" "] # Hierarchy of separators
)
chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks from {len(docs)} documents")
print("\nChunk example: ")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 5 chunks from 3 documents

Chunk example: 
Content: Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through
Metadata: {'source': 'data\\doc_0.txt'}


In [12]:
chunks

[Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of in

### Embedding Model

Used HuggingFace Embedding Models to generate Embeddings

In [25]:
from langchain_huggingface import HuggingFaceEmbeddings
#os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
sample_text = "Maching Learning is Awesome!"
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vector = embeddings.embed_query(sample_text)

In [26]:
vector

[0.020255880430340767,
 -0.05753043666481972,
 0.03902493789792061,
 0.046106334775686264,
 -0.04314039275050163,
 0.0038436874747276306,
 -0.050338730216026306,
 -0.04720974341034889,
 -0.06018934026360512,
 0.06269265711307526,
 0.038995761424303055,
 0.024690257385373116,
 0.005868229549378157,
 0.06480852514505386,
 -0.06965421140193939,
 -0.03188961371779442,
 0.01414196752011776,
 0.030741047114133835,
 -0.006696443073451519,
 -0.03138053044676781,
 -0.014552738517522812,
 -0.04061034694314003,
 0.028637604787945747,
 0.050127025693655014,
 -0.03249882534146309,
 0.09577295184135437,
 -0.03788571059703827,
 -0.01635000668466091,
 0.05438564345240593,
 -0.046161849051713943,
 -0.0006225458928383887,
 0.04173143580555916,
 -0.02527005784213543,
 0.03685855120420456,
 -0.027062086388468742,
 0.008728675544261932,
 0.08921601623296738,
 -0.004677391145378351,
 -0.02543291449546814,
 -0.00766997504979372,
 -0.08191384375095367,
 0.014883884228765965,
 0.03131235018372536,
 -0.04615863

Initialize ChromaDB Vector Store and store the chunks as Vectors